In [5]:
import pandas as pd
import numpy as np
import re #özel karakterleri kaldırmak için
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import words,stopwords
from symspellpy import SymSpell, Verbosity
from nltk.tokenize import word_tokenize
import random

nltk.download('punkt')
nltk.download("words")
nltk.download('stopwords')

correct_words = set(words.words())
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /Users/semih/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/semih/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/semih/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_excel('skillmatrixdata_080923 (2).xlsx')

In [7]:
#Take inputs from users
searched_term = input('What do you need?: ')
#Regularization of inputs
input_query = searched_term.lower().split(' ')
for i in range(len(input_query)):
    input_query[i] = re.sub(r'[^a-zA-Z0-9\s]', '', input_query[i])

input_query = [word for word in input_query if word.lower() not in stop_words]

In [8]:
input_query

['data', 'science']

In [9]:
def suggested_corrections(word):
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    sym_spell.create_dictionary("words.txt")

    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)

    if suggestions:
        return suggestions[0].term
    else:
        return word

In [10]:
suggestions = {}
corrected_word_list = []

for word in input_query:
    if word.lower() not in correct_words:
        correction = suggested_corrections(word)
        suggestions[word] = correction
        corrected_word_list.append(correction)
    else:
        suggestions[word] = word
        corrected_word_list.append(word)

In [11]:
#Check the corrections
for original_word, corrected_word in suggestions.items():
    print(f"You wrote '{original_word}'. Correct form: '{corrected_word}'")

You wrote 'data'. Correct form: 'data'
You wrote 'science'. Correct form: 'science'


In [12]:
corrected_word_list

['data', 'science']

In [13]:
if 'power' and 'bi' in corrected_word_list:
    corrected_word_list.remove('power')
    corrected_word_list.extend(['powerbi','power bi'])
elif 'powerbi' in corrected_word_list:
    corrected_word_list.append('power bi')
elif 'datascience' in corrected_word_list:
    corrected_word_list.extend(['data','science','data science'])
elif 'data' and 'science' in corrected_word_list:
    corrected_word_list.append('data science')
elif 'power' and 'apps' in corrected_word_list:
    corrected_word_list.remove('power')
    corrected_word_list.extend(['powerapps','power apps'])
elif 'data' and 'scientist' in corrected_word_list:
    corrected_word_list.append('data scientist')
elif 'datascientist' in corrected_word_list:
    corrected_word_list.extend(['data','scientist','data scientist'])
elif 'power' and 'apps' in corrected_word_list:
    corrected_word_list.remove('power')
    corrected_word_list.append('powerapps')
elif 'powerapps' in corrected_word_list:
    corrected_word_list.extend(['power apps'])
elif 'powerplatform' in corrected_word_list:
  corrected_word_list.extend(['power platform'])
elif 'power' and 'platform' in corrected_word_list:
  corrected_word_list.remove('power')
  corrected_word_list.append('powerplatform')

In [14]:
corrected_word_list

['data', 'science', 'data science']

In [16]:
#Extend to merged rows
df['Qualifications'] = df['Qualifications'].str.split(',')
df['Qualification Score(0-10)'] = df['Qualification Score(0-10)'].str.split(',')
columns_to_explode = ['Qualifications','Qualification Score(0-10)']
exploded_df = df.explode(columns_to_explode)

In [19]:
exploded_df['Qualification Score(0-10)'] = exploded_df['Qualification Score(0-10)'].astype(float).round()
exploded_df['new_qualifications'] = exploded_df['Qualifications'].str.replace(' ',',')
exploded_df['new_qualifications'] = exploded_df['new_qualifications'].str.lower()
exploded_df['new_position'] = exploded_df['Position'].str.replace(' ',',')
exploded_df['new_position'] = exploded_df['new_position'].str.lower()


AttributeError: Can only use .str accessor with string values!

In [14]:
exploded_df['Score_Qualification'] = exploded_df['new_qualifications'].apply(lambda x: sum(x.count(item.strip()) for item in corrected_word_list))
exploded_df['Position_Score'] = exploded_df['new_position'].apply(lambda x: sum(x.count(item.strip()) for item in corrected_word_list))

In [15]:
last_df = exploded_df[exploded_df['Score_Qualification']!=0]
last_df['Last_Score'] = last_df['Qualification Score(0-10)'] + last_df['Score_Qualification'] + last_df['Position_Score']

<ipython-input-15-e4a9778c0c4d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_df['Last_Score'] = last_df['Qualification Score(0-10)'] + last_df['Score_Qualification'] + last_df['Position_Score']


In [16]:
last_df.head()

Unnamed: 0    Name   Surname                                 Position  \
4          21  Dilara  Kanaralp                     Senior DWH Developer   
5          32   Metin   Türkmen  Junior Business Intelligence Consultant   
6          42    Onur     Soyer                            BI Consultant   
7          49   Caner     Ünsal                            BI Consultant   
7          49   Caner     Ünsal                            BI Consultant   

                       E-Mail NTT LVL Experience Year Qualifications  \
4   dilara.kamci-ext@bshg.com      L3               6       Power BI   
5  metin.turkmen-ext@bshg.com      L1               1       Power BI   
6     onur.soyer-ext@bshg.com      L2               2       Power BI   
7    caner.unsal-ext@bshg.com      L2               2       Power BI   
7    caner.unsal-ext@bshg.com      L2               2           WEBI   

   Qualification Score(0-10)  \
4                        5.0   
5                        4.0   
6                        7.0   
7                        7.0   
7                        4.0   

                        Thema, that can give support  ...      BSH Team  \
4  ETL Processes,Reporting with SSRS,Reporting wi...  ...  BSH_GDS-SEOT   
5  Ability to perform basic/complex queries on re...  ...        AXP-F    
6  Administration, Business Intelligence, Predict...  ...         AXP-F   
7  Visualisation, Power Query,Visualisation,CMC t...  ...         AXP-F   
7  Visualisation, Power Query,Visualisation,CMC t...  ...         AXP-F   

       Shape BSH start date of work  \
4       REM     2021-01-01 00:00:00   
5        DnA    2022-10-25 00:00:00   
6  Analytics    2022-08-08 00:00:00   
7        DnA    2020-04-05 00:00:00   
7        DnA    2020-04-05 00:00:00   

                              CV(with 3-4 sentences)           Person  \
4  I am a software engineer for 6 years. I starte...  Dilara Kanaralp   
5  Data and statistics enthusiast who graduated f...    Metin Türkmen   
6  Business Intelligence Consultant with 2+ years...       Onur Soyer   
7  I graduated from Marmara University, Business ...      Caner Ünsal   
7  I graduated from Marmara University, Business ...      Caner Ünsal   

  new_qualifications                             new_position  \
4           power,bi                     senior,dwh,developer   
5           power,bi  junior,business,intelligence,consultant   
6           power,bi                            bi,consultant   
7           power,bi                            bi,consultant   
7               webi                            bi,consultant   

  Score_Qualification  Position_Score  Last_Score  
4                   1               0         6.0  
5                   1               0         5.0  
6                   1               1         9.0  
7                   1               1         9.0  
7                   1               1         6.0  

[5 rows x 21 columns]

In [17]:
grouped_df = last_df.groupby('Person')['Last_Score'].agg('sum')
grouped_df = grouped_df.reset_index()
grouped_df = grouped_df.sort_values('Last_Score', ascending=False)
grouped_df = grouped_df.reset_index(drop=True)

In [18]:
grouped_df

Person  Last_Score
0       Erhan Kartalkaya        20.0
1            Caner Ünsal        15.0
2           Altan Yıldız        11.0
3       Ayberk Ece Örten        11.0
4            Gamze Çetin        10.0
5          Selma Karaman        10.0
6        Ayşe Kara Şahin         9.0
7        Selim  Kurtuluş         9.0
8             Onur Soyer         9.0
9   Muhammet Ensar  Uslu         9.0
10           Tuna Bozkır         8.0
11      İrem Berfin Ünal         8.0
12          Büşra Sönmez         8.0
13        Ece Hazal Erol         7.0
14           Afrah Ahmed         7.0
15       Dilara Kanaralp         6.0
16         Metin Türkmen         5.0
17       Doğukan Bozkurt         5.0
18       Semih Can Kılıç         5.0
19       Beyza Elif Onat         5.0
20           Hilal Zağlı         4.0
21           Kübra Akyüz         4.0

Danışanların yükleri çok fazla olmasın diye ilk 5 sıradaki kişiler her çalıştırmada rastgele bastırılacaktır.

In [19]:
#Shuffle top-5
grouped_df = grouped_df.iloc[:5]
shuffled_results = grouped_df.sample(frac=1).reset_index(drop=True)

In [20]:
shuffled_results

Person  Last_Score
0       Gamze Çetin        10.0
1  Erhan Kartalkaya        20.0
2      Altan Yıldız        11.0
3  Ayberk Ece Örten        11.0
4       Caner Ünsal        15.0